In [1]:
import sys
sys.path.append('..')
import geedatasets

## Sentinel 2 TOA

In [2]:
s2toa = geedatasets.fromShortName('S2TOA')

In [3]:
print(s2toa)

ID: COPERNICUS/S2
short name: S2TOA
SpaceCraft: Sentinel 2
Process: TOA
Start Date: 2015-06-23
End Date: None
Cloud Cover: CLOUD_COVERAGE_ASSESSMENT
Bands: ['B1 (aerosol)', 'B2 (blue)', 'B3 (green)', 'B4 (red)', 'B5 (red_edge_1)', 'B6 (red_edge_2)', 'B7 (red_edge_3)', 'B8 (nir)', 'B8A (red_edge_4)', 'B9 (water_vapor)', 'B10 (cirrus)', 'B11 (swir)', 'B12 (swir2)', 'QA10 (qa10)', 'QA20 (qa20)', 'QA60 (qa60)']
Masks: ['QA60', 'Hollstein']
Visualizers: ['NSR', 'trueColor', 'falseColor']



### Get the collection, an image and visualize

In [4]:
import ee
ee.Initialize()
import ipygee as ui

In [5]:
Map = ui.Map()
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [6]:
point = ee.Geometry.Point([-72, -42])
Map.centerObject(point, 12)

In [7]:
start = '2019-02-01'
end = '2019-03-01'

In [8]:
collection = s2toa.collection().filterBounds(point)
collection = collection.filterDate(start, end)

In [9]:
image = collection.first()

In [10]:
Map.addLayer(image, s2toa.visualization('trueColor'), 'S2 TOA image trueColor')

## Hollstein mask

In [11]:
mask = s2toa.getMask('Hollstein')
masked = mask.apply(image)

In [12]:
print(mask)

'Hollstein' Mask
options: ['cloud', 'snow', 'shadow', 'water', 'cirrus']
default negatives: ('cloud', 'snow', 'shadow', 'water', 'cirrus')
default positives: None



In [13]:
Map.addLayer(masked, s2toa.visualization('falseColor'), 'S2 TOA image falseColor masked by Hollstein')

In [14]:
# Applied in a map function
filtered = collection.limit(3).map(lambda i: mask.apply(i))
Map.addImageCollection(filtered, s2toa.visualization('falseColor'), 'S2 TOA {system_date} masked by Hollstein', datePattern='y-MM-dd')

## Sentinel 2 SR

In [15]:
s2sr = geedatasets.fromShortName('S2SR')

In [16]:
print(s2sr)

ID: COPERNICUS/S2_SR
short name: S2SR
SpaceCraft: Sentinel 2
Process: SR
Start Date: 2017-03-28
End Date: None
Cloud Cover: CLOUD_COVERAGE_ASSESSMENT
Bands: ['B1 (aerosol)', 'B2 (blue)', 'B3 (green)', 'B4 (red)', 'B5 (red_edge_1)', 'B6 (red_edge_2)', 'B7 (red_edge_3)', 'B8 (nir)', 'B8A (red_edge_4)', 'B9 (water_vapor)', 'B11 (swir)', 'B12 (swir2)', 'AOT (aerosol_thickness)', 'WVP (water_vapor_pressure)', 'SCL (scene_classification_map)', 'TCI_R (true_color_red)', 'TCI_G (true_color_blue)', 'TCI_B (true_color_green)', 'MSK_CLDPRB (cloud_probability)', 'MSK_SNWPRB (snow_probability)', 'QA10 (qa10)', 'QA20 (qa20)', 'QA60 (qa60)']
Masks: ['SCL', 'QA60']
Visualizers: ['NSR', 'trueColor', 'falseColor', 'SCL']



In [17]:
collectionSR = s2sr.collection().filterBounds(point)
collectionSR = collectionSR.filterDate(start, end)

In [18]:
imageSR = collectionSR.first()

In [19]:
Map.addLayer(imageSR, s2sr.visualization('trueColor'), 'S2 SR image trueColor')

### SCL Mask

In [20]:
scl = s2sr.getMask('SCL')
maskedSR = scl.apply(imageSR)

In [21]:
print(scl)

'SCL' Mask
options: ['saturated', 'dark', 'shadow', 'vegetation', 'bare_soil', 'water', 'unclassified', 'clouds_mid_probability', 'clouds_high_probability', 'cirrus', 'snow']
default negatives: ['saturated', 'dark', 'shadow', 'water', 'clouds_mid_probability', 'clouds_high_probability', 'cirrus', 'snow']
default positives: ['vegetation', 'bare_soil', 'unclassified']



In [22]:
Map.addLayer(maskedSR, s2sr.visualization('falseColor'), 'S2 SR masked by SCL falseColor')

In [23]:
# Mask can also be applied in a map function

In [24]:
filteredSR = collectionSR.limit(3).map(lambda i: scl.apply(i))

In [25]:
Map.addImageCollection(filteredSR, s2sr.visualization('NSR'), 'S2 SR {system_date} masked by SCL', datePattern='y-MM-dd')

## Extra bands: ndvi

In [26]:
s2sr.extraBandNames()

['NDVI', 'NBR', 'NDRE']

In [27]:
ndvi = s2sr.getBandByName('NDVI')

In [28]:
print(ndvi)

Band NDVI(ndvi):
resolution: 10
precision: float
units: None


In [29]:
ndvi_i = s2sr.addBand(imageSR, ndvi)

In [30]:
Map.addLayer(ndvi_i, name='S2SR NDVI')

In [31]:
# include extra band in an ImageCollection

In [32]:
ndvi_col = collectionSR.map(lambda i: s2sr.addBand(i, ndvi))

In [33]:
Map.addImageCollection(ndvi_col.limit(3), namePattern='NDVI {system_date}')